# 卷积网络构建

In [12]:
from torch import nn

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        # 提取特征层
        self.features = nn.Sequential(
            # 卷积层
            # 输入图像通道为1，因为使用的是黑白图，单通道的
            # 输出通道为32（代表使用32个卷积核），一个卷积和产生一个单通道的特征图
            # 卷积核kernel_size的尺寸为 3*3， stride代表每次卷积核移动像素个数为1
            # padding填充，为1代表图像长宽都多了两个像素
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
            # 批量归一化，跟上一层的out_channels大小相等，
            nn.BatchNorm2d(num_features=32), # 28*28*32 ---> 28*28*32

            # 激活函数，inplace=true代表直接进行运算
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1), #((28-3+2*1)/1)+1
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),

            # 最大池化层
            # kernel_size 为2*2的滑动窗口
            # stride为2，表示每次滑动距离为2个像素
            # 经过这一步，图像的大小变为1/4，即 28*28 ---> 7*7
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
    
        # 分类层
        self.classifier = nn.Sequential(
            # Dropout层
            # p = 0.5代表该层的每个权重有0.5的可能性为0
            nn.Dropout(p=0.5),
            # 这里是通道数 4* 图像大小 7*7，然后输入到512个神经元中
            nn.Linear(64*7*7, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, 10)
        )


    def forward(self, x):
        #　经过特征提取层
        x = self.features(x)
        # 输出结果必须 展开为 一维向量
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# 模型训练前准备

In [21]:
""" 引入库函数 """
import torch
import matplotlib.pyplot as plt
from torch import nn, optim            # 优化模块，封装了求解模型的一些优化器如Adam,SGD
from torchvision import datasets       # pytorch 视觉库提供了加载数据集的接口
from torchvision import transforms     # pytorch 视觉库中提供了一些数据变换的接口
import torch.nn.functional as F          # 提供了一些常用的函数，如softmax
from torch.optim import lr_scheduler  # 学习率调整器，在训练过程中合理变动学习率
from torch.utils.data import DataLoader

""" 设置超参数 """
# 预设网络超参数 
batch_size = 50 
# 让torch判断是否使用GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
learning_rate = 1e-3

""" 加载数据集 """
data_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])
train_dataset = datasets.MNIST(root='.\data', train=True, 
                               transform=data_transform, download=False)
test_dataset = datasets.MNIST(root='.\data', train=False,
                             transform=data_transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

""" 训练前准备 """
# 初始化模型， 将网格操作移动到GPU或者CPU
ConvModel = ConvNet().to(device)
#　定义交叉熵损失函数
criterion = nn.CrossEntropyLoss().to(device)
#　定义模型优化器
optimizer = optim.Adam(ConvModel.parameters(), lr=learning_rate)
# 定义学习率调度器：输入包装的模型，定义学习率衰减周期step_size，gamma为衰减的乘法因子
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.1)
# 如果学习率lr=0.05, 衰减周期step_size为30，耍贱乘法因子gamma=0.01
# Assuming optimizer uses lr=0.05 for all groups
# >>> # lr = 0.05     if epoch < 30
# >>> # lr = 0.005    if 30 <= epoch < 60
# >>> # lr = 0.0005   if 60 <= epoch < 60

# 训练

In [26]:
def train(num_epochs, _model, _device, _train_loader, _optimizer, _lr_scheduler):
    # 设置模型为训练模式
    _model.train()  
    # 设置学习率调度器开始准备更新
    _lr_scheduler.step()
    for epoch in range(num_epochs):
        # 从迭代器抽取图片和标签
        for i, (images, labels) in enumerate(_train_loader):
            samples = images.to(_device)
            labels = labels.to(_device)
            # 此时样本是一批图片，在CNN的输入中，我们需要将其变为四维，
            # reshape第一个-1 代表自动计算批量图片的数目n
            # 最后reshape得到的结果就是n张图片，每一行图片都是单通道的28*28，得到的四维张量
            output = _model(samples.reshape(-1, 1, 28, 28))
            # 计算损失函数
            loss = criterion(output, labels)
            # 优化器内部参数梯度清零
            optimizer.zero_grad()
            # 损失值后向传播
            loss.backward()
            # 更新模型参数
            optimizer.step()
            if (i + 1) % 100 == 0:
                print("Epoch:{}/{}, step:{}, loss:{:.4f}".format(epoch+1, 
                                            num_epochs, i+1, loss.item()))

# 评估

In [28]:
def test(_test_loader, _model, _device):
    # 设置模型进入评估模式
    _model.eval() 
    loss = 0
    correct = 0
    
    with torch.no_grad(): # 如果不需要 backward更新梯度，那么就要禁用梯度计算，减少内存和计算资源
        for data, target in _test_loader:
            data, target = data.to(_device), target.to(_device)
            output = ConvModel(data.reshape(-1, 1, 28, 28))
            loss += criterion(output, target).item() # 添加损失值
            pred = output.data.max(1, keepdim=True)[1] # 找到概率最大的下标，为输出值
            correct += pred.eq(target.data.view_as(pred)).cpu().sum() # .cpu()是将参数迁移到cpu
            
    loss /= len(_test_loader.dataset)
    
    print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
            loss, correct, len(_test_loader.dataset),
            100.* correct / len(_test_loader.dataset)))
    

# 运行

In [31]:
Epoch = 3
for epoch in range(1, Epoch+1):
    train(epoch, ConvModel, device, train_loader, optimizer, exp_lr_scheduler)
    test(test_loader, ConvModel, device)
    test(train_loader, ConvModel, device)

Epoch:1/1, step:100, loss:0.0455
Epoch:1/1, step:200, loss:0.0519
Epoch:1/1, step:300, loss:0.0519
Epoch:1/1, step:400, loss:0.0773
Epoch:1/1, step:500, loss:0.0092
Epoch:1/1, step:600, loss:0.0796
Epoch:1/1, step:700, loss:0.0026
Epoch:1/1, step:800, loss:0.0143
Epoch:1/1, step:900, loss:0.0509
Epoch:1/1, step:1000, loss:0.0803
Epoch:1/1, step:1100, loss:0.0048
Epoch:1/1, step:1200, loss:0.1821

Average loss: 0.0003, Accuracy: 9963/10000 (99.630%)


Average loss: 0.0002, Accuracy: 59864/60000 (99.773%)

Epoch:1/2, step:100, loss:0.0026
Epoch:1/2, step:200, loss:0.0003
Epoch:1/2, step:300, loss:0.0031
Epoch:1/2, step:400, loss:0.0193
Epoch:1/2, step:500, loss:0.0027
Epoch:1/2, step:600, loss:0.0197
Epoch:1/2, step:700, loss:0.0446
Epoch:1/2, step:800, loss:0.0170
Epoch:1/2, step:900, loss:0.0181
Epoch:1/2, step:1000, loss:0.0020
Epoch:1/2, step:1100, loss:0.0576
Epoch:1/2, step:1200, loss:0.0013
Epoch:2/2, step:100, loss:0.0069
Epoch:2/2, step:200, loss:0.0003
Epoch:2/2, step:300, loss